In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import time
from random import choice
import numpy as np
from datetime import datetime

product = "Lautsprecher"  # for csv and folder
category_name = "hifibox"  # zwischen = und & in url zB hgeschirr60, hgefr, hkuehlsch, hifibox
file_date = "1209"

# User agents list
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/116.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36 Edg/115.0.1901.188"
]

url = f"https://geizhals.at/?cat={category_name}"  # es funzt sowieso nur mit 30 produkten, weil nur 30 auf der seite geladen werden

# Send a GET request to the URL
response = requests.get(url)
time.sleep(np.random.normal(1, 0.2))

# Parse the HTML content
Uebersichtsseite = BeautifulSoup(response.content, "html.parser")
print(Uebersichtsseite.prettify())

<!DOCTYPE HTML>
<html lang="de">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <title>
   HiFi-Komponenten Lautsprecher Preisvergleich Geizhals Österreich
  </title>
  <meta content="//gzhls.at/b/favicons/gh/gh-ogimage.png?ogircp" property="og:image"/>
  <meta content="HiFi-Komponenten Lautsprecher Preisvergleich Geizhals Österreich" property="og:title"/>
  <meta content="Geizhals.at" property="og:site_name"/>
  <meta content="Preisvergleich für HiFi-Komponenten Lautsprecher ✓ Bewertungen ✓ Produktinfos ⇒ Auswahl und Filtern der Produkte nach den besten Eigenschaften und dem billigsten Preis" property="og:description"/>
  <meta content="#005599" media="(prefers-color-scheme: light)" name="theme-color"/>
  <meta content="#161b22" media="(prefers-color-scheme: dark)" name="theme-color"/>
  <!-- geizhals.at=gh_at 3321851;97 -->
  <link href="https://geizhals.at/?cat=hifibox" rel="canonical"/>
  <me

In [2]:
# Save the links for all pages in a list
anzahl_seiten_text = Uebersichtsseite.select_one("#gh_filterbox > div.filtercategory__list-controls > div.sorting-paginator__wrapper > div.paginator__top > nav > ul > li:nth-child(6) > a")
anzahl_seiten = int(re.search(r'\d+', anzahl_seiten_text['aria-label']).group())
print(f"Anzahl Seiten: {anzahl_seiten}")
alle_seiten = [f"https://geizhals.at/?cat={category_name}&pg={i}#productlist" for i in range(1, anzahl_seiten + 1)]

# Create dataframe
variablen_namen = ["Model", "Price_w/o_shipping", "Vendor", "Date"]
data_complete = pd.DataFrame(columns=variablen_namen)

# First loop over all individual pages from ---------Übersichtsseite---------
zaehler = 0
session = requests.Session()

for x, url in enumerate(alle_seiten):
#for x, url in enumerate([f"https://geizhals.at/?cat={category_name}&pg=1#productlist"]):
    print(f"Processing page {x+1}")
    session.headers.update({"User-Agent": choice(user_agents)})
    response = session.get(url)
    time.sleep(np.random.normal(1, 0.2))
    aktuelle_seite = BeautifulSoup(response.content, 'html.parser')

    anzahl_maschinen_text = aktuelle_seite.select_one('#lazy-list--categorylist > div.steel_list_header.productlist__header > div.productlist__item.productlist__name > a')
    anzahl_maschinen = int(anzahl_maschinen_text.text[:anzahl_maschinen_text.text.index(' ')])
    letzte_seite = 30 + anzahl_maschinen - len(alle_seiten) * 30
    maschinen_pro_seite = letzte_seite if x == len(alle_seiten) - 1 else 30

    # Second loop over each machine page ---------Produktseite---------
    for i in range(maschinen_pro_seite):
        try:
            link_node = aktuelle_seite.select_one(f"#product{zaehler} > div.cell.productlist__item.productlist__name > h3 > a")
            if not link_node:
                print(f"Product link not found for index {zaehler}")
                continue
            url_ende = link_node['href']
            aktuelle_url = f"https://geizhals.at/{url_ende}"
            # print(f"Scraping URL: {aktuelle_url}")
            session.headers.update({"User-Agent": choice(user_agents)})
            response = session.get(aktuelle_url)
            time.sleep(np.random.normal(1, 0.2))
            aktueller_link = BeautifulSoup(response.content, 'html.parser')

            anzahl_verkaeufer_text = aktueller_link.select_one("#toc-offercount > span.count")
            anzahl_verkaeufer = int(anzahl_verkaeufer_text.text) if anzahl_verkaeufer_text else 0

            if anzahl_verkaeufer < 5:
                print(f"Skipping product with less than 5 offers: {aktuelle_url}")
                zaehler += 1
                continue

            if zaehler == 2:
                display(data_complete)

            # Third loop over all vendors on the current page and scrape the data we want ---------Vendors---------
            for n in range(anzahl_verkaeufer):
                try:
                    Name = aktueller_link.select_one("#main-content > div.variant.variant--product > div > div.variant__header.card > div.variant__header__details > div.flex.flex--column > h1")
                    Preis_ohne_versand = aktueller_link.select_one(f"#offer-index-{n} > div.offer__price.offer-cell > span")

                    Anbieter = aktueller_link.select_one(f"#offer-index-{n} > div.offer__merchant.offer-cell > div.flex.flex--align-start.gap-1 > a > div.merchant__logo-caption")

                    data_currentseller = {
                        "Model": Name.text.strip() if Name else None,
                        "Price_w/o_shipping": Preis_ohne_versand.text.strip() if Preis_ohne_versand else None,
                        "Vendor": Anbieter.text.strip() if Anbieter else None,
                        "Date": datetime.now().strftime("%Y-%m-%d")
                    }
                    data_complete = pd.concat([data_complete, pd.DataFrame([data_currentseller])], ignore_index=True)

                except Exception as e:
                    print(f"Error processing vendor {n} on product {zaehler}: {e}")
        except Exception as e:
            print(f"Error processing product {zaehler}: {e}")
        print(f"{i+1} machines on page {x+1} processed - {datetime.now().time()}")
        zaehler += 1
        time.sleep(np.random.normal(1, 0.2))

# Save the data in a csv file
data_complete.to_csv(f"{product}/{product}_{file_date}.csv", sep=",", index=True)

Anzahl Seiten: 100
Processing page 1
1 machines on page 1 processed - 11:35:49.399166
Skipping product with less than 5 offers: https://geizhals.at/dali-oberon-5-eiche-hell-a1995217.html?hloc=at


,Model,Price_w/o_shipping,Vendor,Date
0,"Klipsch R-50M, Paar","€ 226,92",Akustik Projekt,2024-09-12
1,"Klipsch R-50M, Paar","€ 259,00",electronic4you,2024-09-12
2,"Klipsch R-50M, Paar","€ 259,00",electronic4you,2024-09-12
3,"Klipsch R-50M, Paar","€ 260,00",HeimkinoWelt,2024-09-12
4,"Klipsch R-50M, Paar","€ 260,99",Media Markt.at,2024-09-12
5,"Klipsch R-50M, Paar","€ 269,00",Majdic,2024-09-12


3 machines on page 1 processed - 11:35:54.101277
Skipping product with less than 5 offers: https://geizhals.at/teufel-ultima-40-aktiv-schwarz-a2013885.html?hloc=at
5 machines on page 1 processed - 11:35:57.254675
6 machines on page 1 processed - 11:35:59.470749
Skipping product with less than 5 offers: https://geizhals.at/focal-theva-no-2-verschiedene-farben-a2971434.html?hloc=at
Skipping product with less than 5 offers: https://geizhals.at/bose-surround-speakers-700-schwarz-834402-2100-a2086756.html?hloc=at
Skipping product with less than 5 offers: https://geizhals.at/teufel-ultima-25-aktiv-night-black-106438001-a3058702.html?hloc=at
Skipping product with less than 5 offers: https://geizhals.at/klipsch-the-fives-schwarz-1068024-a2321733.html?hloc=at
11 machines on page 1 processed - 11:36:06.964679
Skipping product with less than 5 offers: https://geizhals.at/edifier-r1380db-schwarz-r1380db-bk-a2640302.html?hloc=at
13 machines on page 1 processed - 11:36:10.105486
14 machines on page 

In [3]:
display(data_complete)

,Model,Price_w/o_shipping,Vendor,Date
0,"Klipsch R-50M, Paar","€ 226,92",Akustik Projekt,2024-09-12
1,"Klipsch R-50M, Paar","€ 259,00",electronic4you,2024-09-12
2,"Klipsch R-50M, Paar","€ 259,00",electronic4you,2024-09-12
3,"Klipsch R-50M, Paar","€ 260,00",HeimkinoWelt,2024-09-12
4,"Klipsch R-50M, Paar","€ 260,99",Media Markt.at,2024-09-12
...,...,...,...,...
1831,"Vision SP-1900P, Paar","€ 212,90",I-CS,2024-09-12
1832,"Vision SP-1900P, Paar","€ 223,99",visunext.at,2024-09-12
1833,"Vision SP-1900P, Paar","€ 238,90",Future-X.at,2024-09-12
1834,"Vision SP-1900P, Paar","€ 274,80",Schäfer Shop AT,2024-09-12
